# How to use a model to call tools

In [9]:
# https://python.langchain.com/v0.2/docs/how_to/tool_calling/
# Passing tools to chat models
# Chat models that support tool calling features implement a .bind_tools method
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
  """Adds a and b."""
  return a + b

@tool
def multiply(a: int, b: int) -> int:
  """Multiplies a and b."""
  return a * b

tools = [add, multiply]

In [10]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4")

In [11]:
# use the .bind_tools() method to handle converting
llm_with_tools = llm.bind_tools(tools)

In [13]:
# can specify the tools to use with the tool_choice argument
llm_forced_to_multiply = llm.bind_tools(tools, tool_choice="multiply")
llm_forced_to_multiply.invoke("what is 2 + 4")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ICPQYEIISPCr73emrshlQhRm', 'function': {'arguments': '{\n  "a": 2,\n  "b": 4\n}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 84, 'total_tokens': 100}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c8c1155d-a24c-47ee-9968-64e4f1e89a30-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 4}, 'id': 'call_ICPQYEIISPCr73emrshlQhRm'}], usage_metadata={'input_tokens': 84, 'output_tokens': 16, 'total_tokens': 100})

In [14]:
llm_forced_to_use_tool = llm.bind_tools(tools, tool_choice="any")
llm_forced_to_use_tool.invoke("What day is today?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jLPrhqvmLgtAsr1W8lakS1fD', 'function': {'arguments': '{\n  "a": 3,\n  "b": 7\n}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 79, 'total_tokens': 98}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e1ad7457-f45f-43aa-a57a-1a24e5c8662e-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 7}, 'id': 'call_jLPrhqvmLgtAsr1W8lakS1fD'}], usage_metadata={'input_tokens': 79, 'output_tokens': 19, 'total_tokens': 98})

In [16]:
# tool calling

query = "What is 3 * 12? Also, what is 11 + 49?"
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_cAUgLN44vCS1JzWlosWlz9SU'}]

In [17]:
async for chunk in llm_with_tools.astream(query):
  print(chunk.tool_call_chunks)

[{'name': 'multiply', 'args': '', 'id': 'call_8ujbSQ1l5vilIhZQaKMnYgkt', 'index': 0}]
[{'name': None, 'args': '{\n', 'id': None, 'index': 0}]
[{'name': None, 'args': ' ', 'id': None, 'index': 0}]
[{'name': None, 'args': ' "', 'id': None, 'index': 0}]
[{'name': None, 'args': 'a', 'id': None, 'index': 0}]
[{'name': None, 'args': '":', 'id': None, 'index': 0}]
[{'name': None, 'args': ' ', 'id': None, 'index': 0}]
[{'name': None, 'args': '3', 'id': None, 'index': 0}]
[{'name': None, 'args': ',\n', 'id': None, 'index': 0}]
[{'name': None, 'args': ' ', 'id': None, 'index': 0}]
[{'name': None, 'args': ' "', 'id': None, 'index': 0}]
[{'name': None, 'args': 'b', 'id': None, 'index': 0}]
[{'name': None, 'args': '":', 'id': None, 'index': 0}]
[{'name': None, 'args': ' ', 'id': None, 'index': 0}]
[{'name': None, 'args': '12', 'id': None, 'index': 0}]
[{'name': None, 'args': '\n', 'id': None, 'index': 0}]
[{'name': None, 'args': '}', 'id': None, 'index': 0}]
[]
